# Applied Data Science 2 - Keras Assignment - 2023A

In this assignment you will be building a script to classify images of animals. The assignment is broken up into sections and you need to complete each section successively. The sections are:

1. Data Processing
2. Model Definition
3. Model Training
4. Model Evaluation

In addition to this coding exercise, you will also need to write a 1-2 page report analysing and critically evaluating you models results.

In [1]:
# Enter your module imports here, some modules are already provided

import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
import os
import pathlib

In [2]:
# CodeGrade Tag Init1
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Data Processing



In [3]:
# CodeGrade Tag DataProc

label_dict = {'cat' : 0,
              'dog' : 1,
              'wild' : 2}

# This function is provided to read in the image files from the folder on your
# Google Drive
def parse_image(filename):
  parts = tf.strings.split(filename, os.sep)
  label = parts[-2]
  label = tf.strings.to_number(label)

  image = tf.io.read_file(filename)
  image = tf.io.decode_jpeg(image)
  return image, label

img_loc = "/content/drive/MyDrive/Animals/"

train_list_ds = tf.data.Dataset.list_files(img_loc + "train/*/*")
valid_list_ds = tf.data.Dataset.list_files(img_loc + "val/*/*")


**Create a function called "img_process" converts the images to float32 datatype and resizes them to 64x64 pixels.**

In [4]:
# CodeGrade Tag Ex1a
### Write a function called img_process, which takes in the image and label as
### inputs, converts the data type of the image to tf.float32, resizes the
### image to (64,64), and finally returns the image and labels.

import tensorflow as tf
from tensorflow.keras.preprocessing import image

def img_process(image, label, target_size=(64, 64)):
    """
    Preprocesses an image by resizing and converting to float32.

    Parameters:
    - image (tf.Tensor): Input image tensor.
    - label: The label associated with the image.
    - target_size (tuple): Target size for resizing the image.

    Returns:
    - img_array (tf.Tensor): Preprocessed image tensor.
    - label: The input label.
    """

    # Resize the image to the target size
    img_array = tf.image.resize(image, target_size)

    # Convert to float32
    img_array = tf.image.convert_image_dtype(img_array, tf.float32)

    return img_array, label





**Using the tf.data API, load in the training and validation data. Be mindful of efficient data processing good practice to minimise the time it takes to load the data.**

In [5]:
# CodeGrade Tag Ex1b
### Use the parse_image and img_process functions to construct the training and
### validation datasets. You should utilise good practice in optimising the
### dataset loading. Use a batch size of 128. Use techniques like caching and
### prefetching to efficiently load the data.

# Define parse_image function
def parse_image(filename):
    parts = tf.strings.split(filename, os.sep)
    label = parts[-2]
    label = tf.strings.to_number(label)
    image = tf.io.read_file(filename)
    image = tf.io.decode_jpeg(image)
    return image, label

# Define batch size
batch_size = 128

# Create training dataset
train_ds = (
    train_list_ds
    .shuffle(buffer_size=1000)  # Shuffle the dataset
    .map(parse_image, num_parallel_calls=tf.data.experimental.AUTOTUNE)  # Parse the images and labels
    .map(img_process, num_parallel_calls=tf.data.experimental.AUTOTUNE)  # Preprocess the images
    .batch(batch_size)
    .cache()  # Cache the dataset for faster training
    .prefetch(tf.data.experimental.AUTOTUNE)  # Prefetch the next batch while training
)

# Create validation dataset
valid_ds = (
    valid_list_ds
    .map(parse_image, num_parallel_calls=tf.data.experimental.AUTOTUNE)  # Parse the images and labels
    .map(img_process, num_parallel_calls=tf.data.experimental.AUTOTUNE)  # Preprocess the images
    .batch(batch_size)
    .cache()  # Cache the dataset for faster validation
    .prefetch(tf.data.experimental.AUTOTUNE)  # Prefetch the next batch while validating
)

# Model Definition

**Using the Keras Functional API, create a convolutional neural network with the architecture show in the model summary below.**

**A few important points to consider:**

* Call the convolutional layers and the first dense layer should have ReLU activation functions. The output layer should have a SoftMax activation function.
* Pay attention to the output shapes and the number of partmeters for each layer, as these give indications as to the correct settings for the number of filters, kernel size, stride length and padding.
* Use the layer names provided in the summary in your model.


```
# Model Summary

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
=================================================================
 Input (InputLayer)          [(None, 64, 64, 3)]       0         
                                                                 
 Conv0 (Conv2D)              (None, 32, 32, 16)        1216      
                                                                 
 Conv1 (Conv2D)              (None, 32, 32, 32)        4640      
                                                                 
 Conv2 (Conv2D)              (None, 32, 32, 32)        9248      
                                                                 
 Pool1 (MaxPooling2D)        (None, 16, 16, 32)        0         
                                                                 
 Conv3 (Conv2D)              (None, 16, 16, 64)        18496     
                                                                 
 Conv4 (Conv2D)              (None, 16, 16, 64)        36928     
                                                                 
 Pool2 (MaxPooling2D)        (None, 8, 8, 64)          0         
                                                                 
 Conv5 (Conv2D)              (None, 8, 8, 128)         73856     
                                                                 
 Conv6 (Conv2D)              (None, 8, 8, 128)         147584    
                                                                 
 GlobalPool (GlobalAverageP  (None, 128)               0         
 ooling2D)                                                       
                                                                 
 FC1 (Dense)                 (None, 512)               66048     
                                                                 
 Output (Dense)              (None, 10)                5130      
                                                                 
=================================================================
Total params: 363146 (1.39 MB)
Trainable params: 363146 (1.39 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


```



In [6]:
# CodeGrade Tag Ex2a
### Define the model using the Keras Functional API. Use the summary above as a
### guide for the model parameters. You will need to define the filters/units of
### the layers correctly, as well as the kernel size, stride length and padding
### of the convolutional layers.

import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, GlobalAveragePooling2D, Dense

# Define input shape
input_shape = (64, 64, 3)  # Assuming RGB images with size 64x64

# Define input layer
input_layer = Input(shape=input_shape, name="Input")

# Convolutional layers
conv0 = Conv2D(16, (3, 3), activation='relu', padding='same', name="Conv0")(input_layer)
conv1 = Conv2D(32, (3, 3), activation='relu', padding='same', name="Conv1")(conv0)
conv2 = Conv2D(32, (3, 3), activation='relu', padding='same', name="Conv2")(conv1)
pool1 = MaxPooling2D((2, 2), name="Pool1")(conv2)

conv3 = Conv2D(64, (3, 3), activation='relu', padding='same', name="Conv3")(pool1)
conv4 = Conv2D(64, (3, 3), activation='relu', padding='same', name="Conv4")(conv3)
pool2 = MaxPooling2D((2, 2), name="Pool2")(conv4)

conv5 = Conv2D(128, (3, 3), activation='relu', padding='same', name="Conv5")(pool2)
conv6 = Conv2D(128, (3, 3), activation='relu', padding='same', name="Conv6")(conv5)
global_pool = GlobalAveragePooling2D(name="GlobalPool")(conv6)

# Fully Connected layers
fc1 = Dense(512, activation='relu', name="FC1")(global_pool)

# Output layer
output_layer = Dense(3, activation='softmax', name="Output")(fc1)

# Create the model
model = tf.keras.Model(inputs=input_layer, outputs=output_layer)





In [7]:
# CodeGrade Tag Ex2b
### Print the model summary and confirm is has the same architecture as the one
### provided.

# Display the model summary
model.summary()



Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Input (InputLayer)          [(None, 64, 64, 3)]       0         
                                                                 
 Conv0 (Conv2D)              (None, 64, 64, 16)        448       
                                                                 
 Conv1 (Conv2D)              (None, 64, 64, 32)        4640      
                                                                 
 Conv2 (Conv2D)              (None, 64, 64, 32)        9248      
                                                                 
 Pool1 (MaxPooling2D)        (None, 32, 32, 32)        0         
                                                                 
 Conv3 (Conv2D)              (None, 32, 32, 64)        18496     
                                                                 
 Conv4 (Conv2D)              (None, 32, 32, 64)        36928 

**Compile the model using the Adam Optimizer with a learning rate of ```5e-5```, ```sparse categorical crossentropy``` loss function, and ```accuracy``` metric.**

In [8]:
# CodeGrade Tag Ex2c

# Compile the model
optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)
model.compile(optimizer=optimizer,
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])


# Model Training

**Create a Model Checkpoint Callback that saves the weights of the best performing epoch, based on the validation accuracy.**

In [9]:
# CodeGrade Tag Ex3a
### Create a ModelCheckpoint callback to store the bext weights from the model,
### based on the validation accuracy. Call this callback "checkpoint_callback"

from tensorflow.keras.callbacks import ModelCheckpoint

# Define the checkpoint file path
checkpoint_filepath = '/content/checkpoint'

# Create ModelCheckpoint callback
checkpoint_callback = ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_accuracy',  # Monitor validation accuracy
    mode='max',  # Save the model weights with the highest validation accuracy
    save_best_only=True,  # Only save the best model
    verbose=1
)


**Create a Learning Rate Scheduler Callback that utilises the provided function to decrease the learning rate during training.**

In [10]:
# CodeGrade Tag Ex3b
### Using the function provided, create a LearningRateScheduler callback, call
### it "lr_callback"

def scheduler(epoch, lr):
    if epoch < 10:
        return lr
    else:
        return lr * tf.math.exp(-0.01)

from tensorflow.keras.callbacks import LearningRateScheduler

# Create LearningRateScheduler callback
lr_callback = LearningRateScheduler(scheduler, verbose=1)

**Train the model for 100 epochs, using the callbacks you made previously. Store the losses and metrics to use later.**

In [ ]:
# CodeGrade Tag Ex3c
### Train the model for 100 epochs, using the callbacks you have created. Store
### the losses and metrics in a history object.
# Train the model for 100 epochs with callbacks
history = model.fit(
    train_ds,
    epochs=100,
    validation_data=valid_ds,
    callbacks=[checkpoint_callback, lr_callback]
)

# Access training history
training_loss = history.history['loss']
training_accuracy = history.history['accuracy']

# Access validation history
validation_loss = history.history['val_loss']
validation_accuracy = history.history['val_accuracy']


Epoch 1: LearningRateScheduler setting learning rate to 4.999999873689376e-05.
Epoch 1/100
 11/115 [=>............................] - ETA: 36:49 - loss: 1.2756 - accuracy: 0.3438

# Model Evaluation

**Create plots using the losses and metrics. In your report, discuss these results and critically evaluate the models performance.**

In [ ]:
# CodeGrade Tag Ex4a
import matplotlib.pyplot as plt

# Access training history
training_loss = history.history['loss']
training_accuracy = history.history['accuracy']

print(training_loss)
print('\n')
print(training_accuracy)
print('\n')
# Access validation history
validation_loss = history.history['val_loss']
validation_accuracy = history.history['val_accuracy']

print(validation_loss)
print('\n')
print(validation_accuracy)
print('\n')

# Plot training and validation losses
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.plot(training_loss, label='Training Loss')
plt.plot(validation_loss, label='Validation Loss')
plt.title('Training and Validation Losses')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

# Plot training and validation accuracy
plt.subplot(1, 2, 2)
plt.plot(training_accuracy, label='Training Accuracy')
plt.plot(validation_accuracy, label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

plt.show()

**Load the best weights from your model checkpoint, and create plots demonstrating the classification perfomnce for all three classes. Include these plots in your report, and critically evaluate on the performance of the model across the classes.**

In [ ]:
# CodeGrade Tag Ex4b
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np

# Load the best weights from the model checkpoint
model.load_weights(checkpoint_filepath)

# Assuming you have a test dataset (test_ds) defined
# Replace with the actual path to your test dataset
test_list_ds = tf.data.Dataset.list_files("/content/drive/MyDrive/Animals/val/*/*")
test_ds = test_list_ds.map(parse_image).map(img_process).batch(batch_size)

# Make predictions on the test dataset
predictions = model.predict(test_ds)
print('\n')
print(predictions)
print('\n')
# Convert predictions to class labels
predicted_labels = np.argmax(predictions, axis=1)
print('\n')
print(predicted_labels)
print('\n')
# Extract true labels from the dataset
true_labels = np.concatenate([label.numpy() for _, label in test_ds])
print('\n')
print(true_labels)
print('\n')
# Create confusion matrix
conf_matrix = confusion_matrix(true_labels, predicted_labels)
print('\n')
print(conf_matrix)
print('\n')
# Plot confusion matrix
import seaborn as sns
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt='g', cmap='Blues', xticklabels=label_dict.keys(), yticklabels=label_dict.keys())
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.title('Confusion Matrix')
plt.show()

# Create classification report
class_report = classification_report(true_labels, predicted_labels, target_names=label_dict.keys())
print("Classification Report:")
print('\n')
print(class_report)
print('\n')